# Day 16
## Part 1

In [65]:
from dataclasses import dataclass, field
from typing import List


@dataclass 
class Packet:
    version: int
    type_id: int
    
    def version_sum(self):
        return self.version
    
    
@dataclass
class LiteralPacket(Packet):
    value: int
        
        
@dataclass
class OperatorPacket(Packet):
    subpackets: List[Packet] = field(default_factory=list)
        
    def version_sum(self):
        return self.version + sum(sp.version_sum() for sp in self.subpackets)

In [66]:
def hex_to_bin(s):
    return ''.join([f'{int(c, 16):04b}' for c in s])

def read_packet(b, i=0):
    version = int(b[i:i + 3], 2)
    type_id = int(b[i + 3:i + 6], 2)
    
    if type_id == 4:
        bval = ''
        i += 6
        reading_value = b[i]
        while reading_value:
            bval = bval + b[i + 1: i + 5]
            reading_value = b[i] == '1'
            i += 5
        return LiteralPacket(version, type_id, int(bval, 2)), i
    else:
        packet = OperatorPacket(version, type_id)
        length_type = b[i + 6]
        if length_type == '0':
            length = int(b[i + 7:i + 22], 2)
            i += 22
            end = 22 + length
            while i < end:
                subpacket, i = read_packet(b, i)
                packet.subpackets.append(subpacket)
        elif length_type == '1':
            length = int(b[i + 7:i + 18], 2)
            i += 18
            for _ in range(length):
                subpacket, i = read_packet(b, i)
                packet.subpackets.append(subpacket)
        return packet, i
    
def parse_data(s):
    return read_packet(hex_to_bin(s))[0]

parse_data('D2FE28')

LiteralPacket(version=6, type_id=4, value=2021)

In [67]:
parse_data('38006F45291200')

OperatorPacket(version=1, type_id=6, subpackets=[LiteralPacket(version=6, type_id=4, value=10), LiteralPacket(version=2, type_id=4, value=20)])

In [68]:
parse_data('EE00D40C823060')

OperatorPacket(version=7, type_id=3, subpackets=[LiteralPacket(version=2, type_id=4, value=1), LiteralPacket(version=4, type_id=4, value=2), LiteralPacket(version=1, type_id=4, value=3)])

In [69]:
def part_1(data):
    return data.version_sum()

p = parse_data('8A004A801A8002F478')
p

OperatorPacket(version=4, type_id=2, subpackets=[OperatorPacket(version=1, type_id=2, subpackets=[OperatorPacket(version=5, type_id=2, subpackets=[])])])

In [70]:
part_1(p)

10

In [71]:
q = parse_data('620080001611562C8802118E34')
q

OperatorPacket(version=3, type_id=0, subpackets=[OperatorPacket(version=0, type_id=0, subpackets=[LiteralPacket(version=0, type_id=4, value=10)]), LiteralPacket(version=5, type_id=4, value=11)])

In [72]:
r = parse_data('C0015000016115A2E0802F182340')
r

OperatorPacket(version=6, type_id=0, subpackets=[OperatorPacket(version=0, type_id=0, subpackets=[]), LiteralPacket(version=0, type_id=4, value=10), LiteralPacket(version=6, type_id=4, value=11), OperatorPacket(version=4, type_id=0, subpackets=[LiteralPacket(version=7, type_id=4, value=12), LiteralPacket(version=0, type_id=4, value=13)])])

In [73]:
part_1(q)

8

In [74]:
part_1(r)

23